In [3]:
import os 
import numpy as np 
import pandas as pd 
import folium 
import matplotlib.pyplot as plt 

In [4]:
import requests as rq

In [5]:
## making api calls to foursquare
CLIENT_ID = 'DPIZ13PIHP1BYV3EAJVTRO4GPMFPOCBVTHKT45OPNYCKCWGS' # my Foursquare ID
CLIENT_SECRET = 'MMKATV3MMUCFBLIQ4MJELX5ZDD12AFCMCHKJWNI223NRHCIL' #my Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: DPIZ13PIHP1BYV3EAJVTRO4GPMFPOCBVTHKT45OPNYCKCWGS
CLIENT_SECRET:MMKATV3MMUCFBLIQ4MJELX5ZDD12AFCMCHKJWNI223NRHCIL


In [179]:
request_data = {
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "v": '20180605',
    "section":"restaurant",
    "near": "Nairobi",
    "radius": 4000,#within 4km radius
    "limit": 700}

df_nai = rq.get("https://api.foursquare.com/v2/venues/explore", params=request_data)

d = df_nai.json()["response"]


In [162]:
d["geocode"]

{'what': '',
 'where': 'nairobi',
 'center': {'lat': -1.28333, 'lng': 36.81667},
 'displayString': 'Nairobi, Kenya',
 'cc': 'KE',
 'geometry': {'bounds': {'ne': {'lat': -1.18464100563132,
    'lng': 36.93319799789711},
   'sw': {'lat': -1.3330289658680556, 'lng': 36.74527404998827}}},
 'slug': 'nairobi',
 'longId': '72057594038112681'}

In [180]:

items = d["groups"][0]["items"]
print("number of found_: %i" % len(items))
items[8]

number of found_: 100


{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '523c7e7a11d2f542144cccec',
  'name': 'sky lounge bar & bites',
  'location': {'lat': -1.284929592390784,
   'lng': 36.82355971837863,
   'labeledLatLngs': [{'label': 'display',
     'lat': -1.284929592390784,
     'lng': 36.82355971837863}],
   'cc': 'KE',
   'neighborhood': 'City Square',
   'city': 'Nairobi',
   'state': 'Nairobi',
   'country': 'Kenya',
   'formattedAddress': ['Nairobi', 'Kenya']},
  'categories': [{'id': '4bf58dd8d48988d121941735',
    'name': 'Lounge',
    'pluralName': 'Lounges',
    'shortName': 'Lounge',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/nightlife/default_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-523c7e7a11d2f542144cccec-8'}

In [182]:

df_restaurant = []
for item in items:
    venue = item["venue"]
    categories, uid, name,location = venue["categories"], venue["id"], venue["name"],item["venue"]["location"]
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (name, shortname, lat, lng)
    df_restaurant.append(datarow)
df = pd.DataFrame(df_restaurant, columns=["name", "shortname", "lat", "lng"])
print(" %i Restaurants present" % len(df))
df.head()

 100 Restaurants present


,name,shortname,lat,lng
0,CJ's,Coffee Shop,-1.283147,36.818323
1,"Subway,Kenyatta Avenue",Sandwiches,-1.284390,36.821827
2,Cafe Maghreb (Serena),Eastern European,-1.286752,36.814836
3,Najmi Fast Food,Fast Food,-1.280862,36.821209
4,Tatu Restaurant,African,-1.278291,36.816383


In [173]:

# df_restaurant = []
# for item in items:
#     venue = item["venue"]
#     categories, uid, name,location = venue["categories"], venue["id"], venue["name"],item["venue"]["location"]
#     assert len(categories) == 1
#     shortname = categories[0]["shortName"]
#     if not "postalCode" in location:
#         continue
#     lat = location["lat"]
#     lng = location["lng"]
#     postalcode = location["postalCode"]
#     datarow = (name, shortname, postalcode, lat, lng)
#     df_restaurant.append(datarow)
# df = pd.DataFrame(df_restaurant, columns=["name", "shortname", "postalcode", "lat", "lng"])
# print(" %i Restaurants present" % len(df))
# df.head()

 12 Restaurants present


,name,shortname,postalcode,lat,lng
0,Tatu Restaurant,African,00200,-1.278291,36.816383
1,Panda Restaurant,Chinese,00100,-1.285647,36.821439
2,Fairmont Norfolk Hotel,Hotel,200,-1.278407,36.816310
3,Sno-Cream (Kenya House),Ice Cream,GPO,-1.281560,36.817446
4,Sarova Panafric,Hotel,00100,-1.289895,36.810524


In [ ]:
# caitems[2]["venue"]["categories"]

# cat[0]["shortName"]

# items[2]["venue"]["id"]

# items[2]["venue"]["location"]["lat"]

# items[2]["venue"]["location"]["lng"]



In [ ]:
df.head(15)

In [176]:

Nairobi_center = d["geocode"]["center"]
Nairobi_center

from folium import plugins
# create map of Nairobii using latitude and longitude values
map_nairobi = folium.Map(location=[Nairobi_center["lat"], Nairobi_center["lng"]], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='red',
            fill_opacity=0.7,
            parse_html=False).add_to(map_nairobi)

add_markers(df)
hm_data = df[["lat", "lng"]].as_matrix().tolist()
map_nairobi.add_child(plugins.HeatMap(hm_data))

map_nairobi

C:\Users\christopher\Anaconda3\lib\site-packages\ipykernel_launcher.py:23: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [177]:
lat = -1.28333
lng = 36.81667
map_NAI = folium.Map(location=[lat, lng], zoom_start=17)
add_markers(df)
folium.CircleMarker(
    [lat, lng],
    radius=15,
    popup="MAJOR_RESTAURANT_BEST_PLACE",
    color='red',
    fill=True,
    fill_color='yellow',
    fill_opacity=0.7,
    parse_html=False).add_to(map_NAI)
map_NAI